# **Practical No.04**

## **Roll No. 01**
## **Name : Pawan Gosavi**

## **Aim :	Implement Recursive Best-First Search Algorithm for Romanian Map Problem.**


### **P4**

In [1]:
#Import Libraries

!git clone https://gist.github.com/c01f68591ef8ebc0e683bca3a991928d.git myutils
# Clone the entire repo.

# Change directory into cloned repo
%cd myutils

# List repo contents
!ls

# !pip install myutils
# !pip install heapq

from collections import deque
from myutils import *
import sys

infinity = float('inf')

Cloning into 'myutils'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (3/3), done.
/content/myutils
myutils.py


In [2]:
# Class Node
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)
  
    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        return [node.action for node in self.path()[1:]]
        

    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [3]:
# Class Graph
class Graph:
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, A, B, distance=1):
        self.connect1(A, B, distance)
        if not self.directed:
            self.connect1(B, A, distance)

    def connect1(self, A, B, distance):
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):        
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

In [4]:
# Class Problem
class Problem(object):
    def __init__(self, initial, goal=None):
       self.initial = initial
       self.goal = goal

    def actions(self, state):
         raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        return c + 1

    def value(self, state):
        raise NotImplementedError

In [5]:
# Undirected Graph
def UndirectedGraph(graph_dict=None):
    return Graph(graph_dict = graph_dict, directed=False)

In [6]:
# Class Graph Problem
class GraphProblem(Problem):
    def __init__(self, initial, goal, graph):
        Problem.__init__(self, initial, goal)
        self.graph = graph

    def actions(self, A):
        return list(self.graph.get(A).keys())

    def result(self, state, action):
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or infinity)

    def find_min_edge(self):
        m = infinity
        for d in self.graph.graph_dict.values():
            local_min = min(d.values())
            m = min(m, local_min)

        return m

    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(distance(locs[node], locs[self.goal]))

            return int(distance(locs[node.state], locs[self.goal]))
        else:
            return infinity

In [7]:
# BFS
def best_first_graph_search(problem, f):

    f = memoize(f, 'f')    
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    que = []
    ls = []
    ppc = 0
    while frontier:
        print("\n----------------------")
        node = frontier.pop()
        ls.append(node.state)
        # print("\nPopping Node        : " , node)
        print("\nDepth : ", node.depth)
        print("\nGoal Node : ", problem.goal, ", Visiting Node : ", node.state)
        tpc = node.path_cost
        cpc = tpc - ppc
        ppc = cpc
        print("\nCurrent Path Cost: ", cpc)
        if problem.goal_test(node.state):
            print("\nHurrey! Found the Goal! Ending the Search ...")
            print("\nPath : ", ls)
            print("\nTotal Path Cost: ", tpc)
            return node
        x = node.expand(problem)
        print("\nPossible Child Nodes : \t", x)
        explored.add(node.state)
        for child in node.expand(problem):
            # print("\n\tAdding Child Node : \t", child)
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:                
                incumbent = frontier[child]
                print(child , " in frontier. incumbent - ", incumbent)
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
        print("\nQueue : ", explored)
        print("\nPath : ", ls)
        print("\nTotal Path Cost: ", tpc)
    return None

In [8]:
# RBFS Search Algorithm

def recursive_best_first_search(problem, h=None):
    
    h = memoize(h or problem.h, 'h')

    def RBFS(problem, node, flimit):
        if problem.goal_test(node.state):
            return node, 0   # (The second value is immaterial)
        successors = node.expand(problem)
        if len(successors) == 0:
            return None, infinity
        for s in successors:
            s.f = max(s.path_cost + h(s), node.f)
        while True:
            # Order by lowest f value
            successors.sort(key=lambda x: x.f)
            best = successors[0]
            if best.f > flimit:
                return None, best.f
            if len(successors) > 1:
                alternative = successors[1].f
            else:
                alternative = infinity
            result, best.f = RBFS(problem, best, min(flimit, alternative))
            if result is not None:
                return result, best.f
            
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n))

In [9]:
# Romania Map

romania_map = UndirectedGraph({'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
                               'Bucharest': {'Urziceni': 85, 'Pitesti': 101, 'Giurgiu': 90, 'Fagaras': 211},
                               'Craiova': {'Drobeta': 120, 'Rimnicu': 146, 'Pitesti': 138},
                               'Drobeta': {'Mehadia': 75, 'Craiova': 120},
                               'Eforie': {'Hirsova': 86},
                               'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
                               'Hirsova': {'Urziceni': 98, 'Eforie': 86},
                               'Iasi': {'Vaslui': 92, 'Neamt': 87},
                               'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
                               'Mehadia': {'Drobeta': 75, 'Lugoj': 70},
                               'Neamt': {'Iasi': 87},
                               'Oradea': {'Zerind': 71, 'Sibiu': 151},
                               'Pitesti': {'Rimnicu': 97, 'Bucharest': 101, 'Craiova': 138},
                               'Rimnicu': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
                               'Sibiu': {'Arad': 140, 'Fagaras': 99, 'Oradea': 151, 'Rimnicu': 80},
                               'Timisoara': {'Arad': 118, 'Lugoj': 111}, 'Giurgiu': {'Bucharest': 90},
                               'Urziceni': {'Vaslui': 142, 'Bucharest': 85, 'Hirsova': 98},
                               'Vaslui': {'Iasi': 92, 'Urziceni': 142},
                               'Zerind': {'Arad': 75, 'Oradea': 71}})

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

In [10]:
# Graph

print("\nThis is the Graph - \n")

romania_map.graph_dict


This is the Graph - 



{'Arad': {'Sibiu': 140, 'Timisoara': 118, 'Zerind': 75},
 'Bucharest': {'Fagaras': 211, 'Giurgiu': 90, 'Pitesti': 101, 'Urziceni': 85},
 'Craiova': {'Drobeta': 120, 'Pitesti': 138, 'Rimnicu': 146},
 'Drobeta': {'Craiova': 120, 'Mehadia': 75},
 'Eforie': {'Hirsova': 86},
 'Fagaras': {'Bucharest': 211, 'Sibiu': 99},
 'Giurgiu': {'Bucharest': 90},
 'Hirsova': {'Eforie': 86, 'Urziceni': 98},
 'Iasi': {'Neamt': 87, 'Vaslui': 92},
 'Lugoj': {'Mehadia': 70, 'Timisoara': 111},
 'Mehadia': {'Drobeta': 75, 'Lugoj': 70},
 'Neamt': {'Iasi': 87},
 'Oradea': {'Sibiu': 151, 'Zerind': 71},
 'Pitesti': {'Bucharest': 101, 'Craiova': 138, 'Rimnicu': 97},
 'Rimnicu': {'Craiova': 146, 'Pitesti': 97, 'Sibiu': 80},
 'Sibiu': {'Arad': 140, 'Fagaras': 99, 'Oradea': 151, 'Rimnicu': 80},
 'Timisoara': {'Arad': 118, 'Lugoj': 111},
 'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
 'Vaslui': {'Iasi': 92, 'Urziceni': 142},
 'Zerind': {'Arad': 75, 'Oradea': 71}}

In [11]:
# Map Loactions

print("\nThis is the Graph Coordinates - \n")

romania_map.locations


This is the Graph Coordinates - 



{'Arad': (91, 492),
 'Bucharest': (400, 327),
 'Craiova': (253, 288),
 'Drobeta': (165, 299),
 'Eforie': (562, 293),
 'Fagaras': (305, 449),
 'Giurgiu': (375, 270),
 'Hirsova': (534, 350),
 'Iasi': (473, 506),
 'Lugoj': (165, 379),
 'Mehadia': (168, 339),
 'Neamt': (406, 537),
 'Oradea': (131, 571),
 'Pitesti': (320, 368),
 'Rimnicu': (233, 410),
 'Sibiu': (207, 457),
 'Timisoara': (94, 410),
 'Urziceni': (456, 350),
 'Vaslui': (509, 444),
 'Zerind': (108, 531)}

In [12]:
print("\n          --/--\--/  RBFS Search Algorithm  \--/--\-- ")
StartNode = input ("\nEnter the Node from where You want to Start :\t")
EndNode = input ("\nEnter Your Goal Node where You want to go   :\t")


          --/--\--/  RBFS Search Algorithm  \--/--\-- 

Enter the Node from where You want to Start :	Arad

Enter Your Goal Node where You want to go   :	Bucharest


In [13]:
print ("\nStart Node :\t", StartNode)
print ("\nGoal Node :\t", EndNode)

romania_problem = GraphProblem(StartNode, EndNode, romania_map)


Start Node :	 Arad

Goal Node :	 Bucharest


In [14]:
finalnode = recursive_best_first_search(romania_problem)


----------------------

Depth :  0

Goal Node :  Bucharest , Visiting Node :  Arad

Current Path Cost:  0

Possible Child Nodes : 	 [<Node Zerind>, <Node Sibiu>, <Node Timisoara>]

Queue :  {'Arad'}

Path :  ['Arad']

Total Path Cost:  0

----------------------

Depth :  1

Goal Node :  Bucharest , Visiting Node :  Sibiu

Current Path Cost:  140

Possible Child Nodes : 	 [<Node Arad>, <Node Fagaras>, <Node Oradea>, <Node Rimnicu>]

Queue :  {'Arad', 'Sibiu'}

Path :  ['Arad', 'Sibiu']

Total Path Cost:  140

----------------------

Depth :  2

Goal Node :  Bucharest , Visiting Node :  Fagaras

Current Path Cost:  99

Possible Child Nodes : 	 [<Node Sibiu>, <Node Bucharest>]

Queue :  {'Fagaras', 'Arad', 'Sibiu'}

Path :  ['Arad', 'Sibiu', 'Fagaras']

Total Path Cost:  239

----------------------

Depth :  2

Goal Node :  Bucharest , Visiting Node :  Rimnicu

Current Path Cost:  121

Possible Child Nodes : 	 [<Node Sibiu>, <Node Craiova>, <Node Pitesti>]

Queue :  {'Fagaras', 'Arad', '

In [15]:
if finalnode == None:
    print("Sorry! No Path was Found!")
else:
    rbfspath = []
    for i in finalnode.path():
        rbfspath.append(i.state)
        
    print("\nRBFS")
    print("\nStart Node : \t", romania_problem.initial)
    print("\nGoal Node  : \t", romania_problem.goal)
    print("\nPath       : \t", rbfspath)
    print("\nPath Cost  : \t", finalnode.path_cost)
    print("\nDepth      : \t", finalnode.depth)


RBFS

Start Node : 	 Arad

Goal Node  : 	 Bucharest

Path       : 	 ['Arad', 'Sibiu', 'Rimnicu', 'Pitesti', 'Bucharest']

Path Cost  : 	 418

Depth      : 	 4
